# ETL Steam Games

Este notebook presenta la extracción, transformación y carga de el data frame de juegos de steam, junto con los criterios de por que se modifico, conservó o eliminó cada parte de la información.

---

# 1. Importamos las librerías que usaremos.

In [21]:
import pandas as pd # Usaremos pandas como nuestra librería principal para trabajar con todo lo relacionado a Datos.
import re # Expresiones regulares para buscar texto de una manera más óptima.
import numpy as np # Librería para trabajar con matrices.
from difflib import SequenceMatcher # Con este objeto podremos encontrar similitudes entre cadenas de texto similares.
import os # Trabajamos en nuestro sistema operativo con OS.
import gdown # Descargaremos archivos que no pudieron ser subidos al git hub desde google drive.

# 2. Carga de datos.

Verificamos que el DataFrame exista en nuestro equipo.

In [22]:
def verifica(path, driv):
    # Comprobamos que el archivo exista.
    if not os.path.isfile(path):
        # Si el archivo no existe, lo bajamos de google drive.
        gdown.download(driv, path)
        print("Descargado correctamente")
    elif os.path.isfile(path):
        print("El archivo ya está presente.")
    else:
        print("No se encontró el archivo en drive.")

In [23]:
path = '../Data/steam_games.json.gz'
driv = 'https://drive.google.com/uc?id=18PU0Me1MkEusikw9wucNtR2jkHusVtvG'
verifica(path, driv)

El archivo ya está presente.


Cargamos el primer data set que usaremos para nuestra tarea. El de juegos de Steam, y lo previsualizamos.

In [24]:
df_games = pd.read_json(path, lines=True)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


Observamos una gran cantidad de valores nulos a primera vista, lo que indica que nos podemos quedar con muy pocos datos para nuestro objetivo, en otro caso sería mejor explorar otras opciones de data sets que contengan esta misma información pero sin tantos valores nulos, también verificar que pandas esté cargando correctamente el archivo json, lo cual ya verifiqué y el json en efecto tiene todos esos valores nulos. Sin embargo en este caso usaremos este data set e intentaremos ser muy cuidadosos a la hora de la limpieza para aprovechar al máximo los datos que tenemos.

---

# 3. Primera vista y limpieza.

Observemos los valores nulos para saber que tantos hay respecto al total de registros ( 120,445 )

In [25]:
df_games.isna().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Vemos que al menos 96,362 de los 120,445 valores que tenemos son nulos, empecemos eliminando las filas que contengan unicamente valores nulos y veamos cuantos valores nos quedan para trabajar.

In [26]:
df_games = df_games.dropna(how='all').reset_index(drop=True)

In [27]:
print(df_games.shape)
df_games.isna().sum()

(32135, 13)


publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

Verificamos y eliminamos los duplicados

In [28]:
df_games[df_games.duplicated(subset=['id'])]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [29]:
df_games = df_games.drop_duplicates(subset=['id'])

Observamos que la columna publisher es la que más valores nulos tiene, sin embargo, la columna developer no tiene tantos y la mayoría de desarrolladores sin publisher publican sus propios juegos, también esto nos será útil ya que el publisher no será usado en nuestras API por lo que no entorpecerá los resultados, y a su vez, puede beneficiar al algoritmo de recomendación, ya que varios usuarios pueden preferir a algunos publishers para elegir sus juegos, y por último también será útil para salvar algunos registros, ya que nos quedaron pocos datos al hacer el primer drop.

In [30]:
df_games[df_games['publisher'].isna() & df_games['developer'].notna()].shape # Primero observamos cuantos registros podemos salvar al usar al developer como publisher.

(4818, 13)

Al ver que son 4,818 registros los que se pueden salvar, ejecutamos el plan y visualizamos el resultado.

In [31]:
df_games['publisher'] = df_games['publisher'].fillna(df_games['developer'])
df_games.isna().sum()

publisher       3234
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        1
specs            670
price           1377
early_access       0
id                 1
developer       3299
dtype: int64

El número máximo de nulos ahora se redujo a 3 mil, podemos seguir observando qué otras columnas comparten características y nos pueden ahorrar la eliminación de filas. En este caso, las columnas `app_name` y `title` pueden tener similitudes.

In [32]:
print('AppName >>>\n',df_games['app_name'].tail(), '\n-----------------------------------\nTitle >>>', sep='')
df_games['title'].tail()

AppName >>>
32130              Colony On Mars
32131    LOGistICAL: South Africa
32132               Russian Roads
32133         EXIT 2 - Directions
32134                 Maze Run VR
Name: app_name, dtype: object
-----------------------------------
Title >>>


32130              Colony On Mars
32131    LOGistICAL: South Africa
32132               Russian Roads
32133         EXIT 2 - Directions
32134                        None
Name: title, dtype: object

Las columnas, en efecto comparten mucha similitud, por lo que nos quedaremos con la que tenga menos valores nulos y la nombraremos como name.

In [33]:
df_games = df_games.drop(columns=['title']).reset_index(drop=True)
df_games = df_games.rename(columns={'app_name': 'name'})
df_games.head()

,publisher,genres,name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
4,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


---

Observamos si podemos encontrar más columnas similares

In [34]:
df_games.isna().sum()

publisher       3234
genres          3283
name               2
url                0
release_date    2067
tags             163
reviews_url        1
specs            670
price           1377
early_access       0
id                 1
developer       3299
dtype: int64

---

Tanto `specs` como `tags` y `genres` parecen tener una similitud, procedemos a compararlas.

In [35]:
print('tags >>>\n',df_games['tags'].tail(), '\n-----------------------------------------------------------------------', sep='')
print('genres >>>\n',df_games['genres'].tail(), '\n-----------------------------------------------------------------------\nspecs >>>', sep='')
df_games['specs'].tail()

tags >>>
32128                [Strategy, Indie, Casual, Simulation]
32129                            [Strategy, Indie, Casual]
32130                          [Indie, Simulation, Racing]
32131    [Indie, Casual, Puzzle, Singleplayer, Atmosphe...
32132    [Early Access, Adventure, Indie, Action, Simul...
Name: tags, dtype: object
-----------------------------------------------------------------------
genres >>>
32128    [Casual, Indie, Simulation, Strategy]
32129                [Casual, Indie, Strategy]
32130              [Indie, Racing, Simulation]
32131                          [Casual, Indie]
32132                                     None
Name: genres, dtype: object
-----------------------------------------------------------------------
specs >>>


32128                  [Single-player, Steam Achievements]
32129    [Single-player, Steam Achievements, Steam Clou...
32130    [Single-player, Steam Achievements, Steam Trad...
32131     [Single-player, Steam Achievements, Steam Cloud]
32132    [Single-player, Stats, Steam Leaderboards, HTC...
Name: specs, dtype: object

Únicamente tags y genres comparten características, por lo que podemos usar los valores presentes en tags para llenar en genres nuevamente, y luego desechar la columna tags.

In [36]:
df_games['genres'] = df_games['genres'].fillna(df_games['tags']) # Llenamos los valores faltantes en genres con los presentes en tags
df_games = df_games.drop(columns=['tags']).reset_index(drop=True) # Eliminamos la columna tags que ya no usaremos.
df_games.head() # Previsualizamos

,publisher,genres,name,url,release_date,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
4,None,"[Action, Indie, Casual, Sports]",Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


---

Ahora borramos las url que no las usaremos para una mejor visualización.

In [37]:
df_games = df_games.drop(columns=['url', 'reviews_url']).reset_index(drop=True)

In [38]:
df_games.isna().sum()

publisher       3234
genres           139
name               2
release_date    2067
specs            670
price           1377
early_access       0
id                 1
developer       3299
dtype: int64

Procedemos a eliminar los valores faltantes que son esenciales para ambos objetivos (La API y el algoritmo de recomendación) y que ya no podemos conseguir de ningún otra fuente.

Para esto usaré el siguiente criterio:

1. No borraré los registros de las columnas que sean requeridos unicamente en ciertas funciones de la API, por ejemplo, developer, es requerido en 3 de 5 funciones de la API, por lo que no vale la pena borrar sus registros faltantes para este caso, pero sí para el algoritmo de recomendación y el EDA. Lo mismo ocurre con genres, price, etc...

2. La columna specs no se usa en ningún lugar de la API, por lo que en el data frame resultante para la API no la usaré, pero puede ser muy útil para el algoritmo de recomendación. Lo mismo ocurre con la columna publisher.

In [39]:
# Empiezo borrando los esenciales para ambos objetivos (API y Algoritmo).
df_games = df_games.dropna(subset=[ 'id', # El id tiene unicamente 2 nan, uno que no tiene registros y otro repetido, por lo que lo eliminamos.
                                    'name' # El nombre del juego también es indispensable y tiene unicamente 2 valores faltantes.
                                ]).reset_index(drop=True)

Convertimos el id a string para su futuro uso con otros dataframes.

In [40]:
df_games['id'] = df_games['id'].astype(int).astype(str)

---

# 4. Transformación.

Por último procesamos la columna release date, de la cual solo necesitaremos el año y convertimos el Free To Play de price en 0 para su uso aritmético.

In [41]:
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce') # Convertimos la columna a fecha y evitamos que el programa se corte si alguna fecha no puede convertirse.
df_games['year'] = df_games['release_date'].dt.year # Extraemos el año.
df_games = df_games.drop(columns=['release_date']).reset_index(drop=True) # Eliminamos la columna de fecha de salida ya que no la usaremos más.

Seguimos con el precio

In [42]:
def precio(price):
    # Primero verificamos que el precio no sea un nulo, si lo es, lo dejamos nulo.
    if price is None:
        return np.nan
    # Si es numérico, lo dejamos como está.
    elif isinstance(price, (int, float)):
        return price
    # Si es un string, buscamos la palabra free, de free to play, si la encuentra, será 0 su precio.
    elif re.search(r'\bFree\b', price):
        return 0
    # Si no logra hacer nada de lo anterior, el precio será nulo, en palabras desconocidas o errores.
    else:
        return np.nan

df_games['price'] = df_games['price'].apply(precio)

Convertimos la columna a flotante para su futuro uso, al igual que el año a objeto.

In [43]:
df_games['price'] = df_games['price'].astype(float) # El precio lo volvemos flotante.
df_games['year'] = df_games['year'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan) # El año lo volvemos string si no es None, si es None lo dejamos como está.

In [44]:
df_games.head() # Previsualizamos

,publisher,genres,name,specs,price,early_access,id,developer,year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,[Single-player],4.99,0.0,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,[Single-player],0.99,0.0,767400,彼岸领域,2017
4,None,"[Action, Indie, Casual, Sports]",Log Challenge,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570,None,NaN


La columna de desarrolladores debemos revisarla para saber si hay algún error en la escritura de estos, como por ejemplo Valve se pueda llamar de distintas formas, como valve. Valve. valvE, etc...

In [45]:
# Extraemos los nombres de cada desarrollador, excluyendo los nulos, y ordenándolos para no iterar en todas las filas, ya que cada fila debe iterar sobre todas,
# y eso es demasiado proceso para una lista tan grande, al estar la lista ordenada, podemos solo recorrer los primeros elementos, que es donde notoriamente va a haber una similitud.
dev_names = (sorted(df_games['developer'].dropna().unique())) # creamos un df temporal, ordenado, sin nulos, y retornamos sus valores únicos.
dev_names = pd.DataFrame({'developer': dev_names}) # Creamos un df en base a la lista de desarrolladores únicos.
# Con esta función encontraremos el puntaje que le da el SequenceMatcher basando en la similitud de las cadenas.
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio() # Retornamos la similitud entre las cadenas de texto A y B.
# Con esta función agrupamos las cadenas de texto similares, el threshold lo establecemos en 0.93 para que sea muy estricto, no queremos cambiar
# cadenas similares por accidente, en especial en este dataframe que muchos estudios tienen palabras similares como 'studios' o 'Games', etc...
# Modificar el valor de threshold para resultados diferentes.
def agrupar_texto(df, threshold=0.93):
    # Guardamos en un diccionario los desarrolladores que se lograron agrupar.
    agrupados = {}
    # Guardamos los ya categorizados para evitar redundancias.
    done = []
    # Iteramos en el indice y las filas de el dataframe ingresado en la función.
    for i, row in df.iterrows():
        # Limitar la comparación solo a los siguientes 10 registros.
        for j in range(i + 1, min(i + 11, len(df))):
            nombre = df.loc[j, 'developer']
            similitud = similar(row['developer'], nombre)
            if similitud > threshold:
                # print( row['developer'],' ==== ', nombre, '--',similitud)
                # Agregar el desarrollador al grupo encontrado, si no existe el grupo, se crea.
                if not(nombre in done):
                    agrupados.setdefault(row['developer'], []).append(nombre)
                done.append(nombre)
    return agrupados

# Agrupamos los desarrolladores con nuestra función.
similar_devs = agrupar_texto(dev_names)
# Visualizamos los encontrados.
# Cambiar el valor de limite para ver mas o menos registros, para no saturar el notebook con texto, solo mostraré los primeros 10 como ejemplo.
limite = 10
c = 0
# Contamos cuantos registros detectó el algoritmo
print(f"Hay {len(similar_devs)} registros similares. Ejemplo de los {limite} primeros:\n")
for dev, grupo in similar_devs.items():
    c += 1
    print(f"{dev} -----> {grupo}")
    if c == limite:
        break

Hay 119 registros similares. Ejemplo de los 10 primeros:

14 Dimension Enterprise -----> ['14Dimension Enterprise']
2Chance Projects,IIchan Eroge Team -----> ['2Chance Projects,IIchan Eroge Team,DjSM']
2K Australia,Gearbox Software,Aspyr (Mac &amp; Linux) -----> ['2K Australia,Gearbox Software,Aspyr (Mac and Linux)', '2K Australia,Gearbox Software,Aspyr (Mac, Linux)']
2K Australia,Gearbox Software,Aspyr (Mac and Linux) -----> ['2K Australia,Gearbox Software,Aspyr (Mac,Linux)']
AK Games -----> ['AKGames']
Absolutist Ltd -----> ['Absolutist Ltd.']
Accolade, Inc -----> ['Accolade, Inc.']
Alexander Shvab -----> ['Alexandr Shvab']
Arrowhead Game Studio -----> ['Arrowhead Game Studios']
Artefacts Studio -----> ['Artefacts Studios']


Ya teniendo el diccionario de los nombres que debemos reemplazar, lo usamos en un script para reemplazar los nombres de los desarrolladores con un mapeo.

In [46]:
# contamos los registros reemplazados
c = 0
# Iteramos en cada desarrollador, y en su grupo de similitudes.
for dev, grupo in similar_devs.items():
    # Iteramos en cada elemento del grupo.
    for elm in grupo:
        # Si se encuentra al developer en el grupo de repetidos, lo reemplaza por dev, el original.
        if elm in df_games['developer'].values:
            df_games.replace(elm, dev, inplace=True) # Reemplazo.
            c+=1
            #print(f'replaced {df_games.loc[elm, 'developer']} for {dev}') # debug
print(f'Reemplazados {c} registros')
df_games

Reemplazados 120 registros


,publisher,genres,name,specs,price,early_access,id,developer,year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,[Single-player],4.99,0.0,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,[Single-player],0.99,0.0,767400,彼岸领域,2017
4,None,"[Action, Indie, Casual, Sports]",Log Challenge,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570,None,NaN
...,...,...,...,...,...,...,...,...,...
32126,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,"[Single-player, Steam Achievements]",1.99,0.0,773640,"Nikita ""Ghost_RUS""",2018
32127,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530,Sacada,2018
32128,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660,Laush Dmitriy Sergeevich,2018
32129,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870,"xropi,stev3ns",2017


Por último elimino los / que pueda tener la columna de búsqueda, ya que puede dar errores al las URL usar /.

In [47]:
df_games['developer'] = df_games['developer'].apply(lambda x: x.replace('/', '') if x else x)

---

Ahora creamos el data frame que usaremos para la API, el cual ignorará las columnas que no se usen en ninguna de estas y mantendrá los valores nulos que no se usen en todas las funciones de esta, como mencioné en el punto 1 de los criterios.

In [48]:
df_api = df_games.drop(columns=['publisher', 'specs', 'early_access'])

---

Vemos como quedó el df que usaremos en la API.

In [49]:
df_api.isna().sum()

genres        138
name            0
price        1386
id              0
developer    3297
year         2351
dtype: int64

Eliminamos los faltantes que sean al menos faltantes en 3 columnas, ya que estos nos servirán muy poco en la API.

In [50]:
df_api = df_api.dropna(subset=['price', 'developer', 'year'], how='all')

In [51]:
df_api.isna().sum()

genres        138
name            0
price        1270
id              0
developer    3181
year         2235
dtype: int64

In [52]:
df_api.head()

,genres,name,price,id,developer,year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,0.00,643980,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400,彼岸领域,2017
4,"[Action, Indie, Casual, Sports]",Log Challenge,2.99,773570,None,NaN


Lo guardamos listo para su uso.

# 5. Primera carga.

In [53]:
df_api.to_csv('../Data/Processed/games_api.csv', index=False)

In [54]:
del df_api # borramos el df para ahorrar memoria en el computador.

---
---

# 6. Data frame para algoritmo de recomendación.

Ahora repetimos el proceso de tratar el dataframe que será usado para el algoritmo de recomendación. Empecemos descartando las columnas que no creamos útiles.

In [55]:
df_games.isna().sum()

publisher       3232
genres           138
name               0
specs            669
price           1386
early_access       0
id                 0
developer       3297
year            2351
dtype: int64

No consideré ninguna columna en este caso para el descarte, la más cercana sería el año, pero hay usuarios que tal vez prefieran los juegos actuales respecto a los viejos, entonces usaré todas las columnas disponibles, eliminaré los valores faltantes para no afectar al análisis exploratorio de datos que haré antes de hacer el algoritmo y guardaré el archivo para su uso.

In [56]:
df_games = df_games.dropna().reset_index(drop=True) # Eliminamos los faltantes.

In [57]:
df_games.isna().sum() # Verificamos.

publisher       0
genres          0
name            0
specs           0
price           0
early_access    0
id              0
developer       0
year            0
dtype: int64

# 7. Segunda carga.

In [58]:
df_games.to_csv('../Data/Processed/games_recommend.csv', index=False) # Guardamos el archivo listo para el EDA.